In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Inverse Gamma

In [68]:
prior_models_invgamma=dict()
particles_invgamma=dict()

In [70]:
c0 = 10
model_code = """
data {
  int c0;
}

generated quantities{
  vector<lower=0>[1] sigma;
  vector<lower=0>[1] sigma_square;
  sigma_square[1] = inv_gamma_rng(c0,c0);
  sigma = sqrt(sigma_square);
}
"""

sm = pystan.StanModel(model_code=model_code, verbose=False)
# save_obj(sm, "sm_prior", log_dir)
prior_models_invgamma[c0] = sm

param_names = ['sigma']
fit_run = prior_models_invgamma[c0].sampling(
    data={'c0':c0},
    iter=100,
    warmup=0,
    chains=1,
    algorithm="Fixed_param",
    n_jobs=1
)

particles_invgamma[c0] = fit_run.extract(permuted=False, pars=param_names)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ef0acf620f81d6f78fce9098e0530704 NOW.


In [71]:
param = 'sigma'
dfs = []
for c0 in [1,10]:
    df = get_post_df(np.squeeze(particles_invgamma[c0][param]))
    df['source'] = str(c0)
    dfs.append(df)
plot_density(pd.concat(dfs), 500, 200).interactive()

alt.Chart(...)

In [73]:
from scipy.stats import invgamma
c0 = 10

dfs = []
for c0 in [1,10]:
    df = get_post_df(np.sqrt(invgamma.rvs(a = c0, scale=c0, size=100)))
    df['source'] = str(c0)
    dfs.append(df)
plot_density(pd.concat(dfs), 500, 200).interactive()

alt.Chart(...)

## Check Half-Cauchy prior

In [55]:
prior_models_cauchy=dict()
particles_cauchy=dict()


In [63]:
c0 = 5
model_code = """
data {
  int c0;
}

generated quantities{
  vector<lower=0>[1] sigma;
  sigma[1] = abs(cauchy_rng(0,c0));
}
"""

sm = pystan.StanModel(model_code=model_code, verbose=False)
# save_obj(sm, "sm_prior", log_dir)
prior_models_cauchy[c0] = sm

param_names = ['sigma']
fit_run = prior_models_cauchy[c0].sampling(
    data={'c0':c0},
    iter=100,
    warmup=0,
    chains=1,
    algorithm="Fixed_param",
    n_jobs=1
)

particles_cauchy[c0] = fit_run.extract(permuted=False, pars=param_names)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8cec7349647c01e13ab6de1eca9ce14a NOW.


In [65]:
c0=10
param = 'sigma'
dfs = []
for c0 in [1,5,10]:
    df = get_post_df(np.squeeze(particles_cauchy[c0][param]))
    df['source'] = str(c0)
    dfs.append(df)
plot_density(pd.concat(dfs), 500, 200).interactive()

alt.Chart(...)

In [74]:
prior_models_uniform=dict()
particles_uniform=dict()


In [75]:
c0 = 5
model_code = """
data {
  int c0;
}

generated quantities{
  vector<lower=0>[1] sigma;
  sigma[1] = uniform_rng(0,c0);
}
"""

sm = pystan.StanModel(model_code=model_code, verbose=False)
# save_obj(sm, "sm_prior", log_dir)
prior_models_uniform[c0] = sm

param_names = ['sigma']
fit_run = prior_models_uniform[c0].sampling(
    data={'c0':c0},
    iter=100,
    warmup=0,
    chains=1,
    algorithm="Fixed_param",
    n_jobs=1
)

particles_uniform[c0] = fit_run.extract(permuted=False, pars=param_names)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0f874ca20329c9e1fe969afa61bdc9ca NOW.


In [76]:
param = 'sigma'
dfs = []
for c0 in [5]:
    df = get_post_df(np.squeeze(particles_uniform[c0][param]))
    df['source'] = str(c0)
    dfs.append(df)
plot_density(pd.concat(dfs), 500, 200).interactive()

alt.Chart(...)